In [1]:
import os
import shutil
from glob import glob

In [2]:
def delete_files(expr):
    _lis=glob(expr)
    for i in _lis:
        os.remove(i)
    print(f'delete {len(_lis)} files')

In [3]:
new_dest='D:\\春歌ナナVer1.3\\New'
if not os.path.exists(new_dest):
    os.mkdir(new_dest)

In [4]:
#使用后缀排除当前目录下新文件夹
lis=glob('D:/春歌*/*.*')
print(len(lis))
lis

288


['D:/春歌ナナVer1.3\\character.txt',
 'D:/春歌ナナVer1.3\\image.bmp',
 'D:/春歌ナナVer1.3\\oto-clean.ini',
 'D:/春歌ナナVer1.3\\oto.ini',
 'D:/春歌ナナVer1.3\\Thumbs.db',
 'D:/春歌ナナVer1.3\\_あb.wav',
 'D:/春歌ナナVer1.3\\_あb_wav.frq',
 'D:/春歌ナナVer1.3\\_いb.wav',
 'D:/春歌ナナVer1.3\\_いb_wav.frq',
 'D:/春歌ナナVer1.3\\_うb.wav',
 'D:/春歌ナナVer1.3\\_うb_wav.frq',
 'D:/春歌ナナVer1.3\\_えb.wav',
 'D:/春歌ナナVer1.3\\_えb_wav.frq',
 'D:/春歌ナナVer1.3\\_おb.wav',
 'D:/春歌ナナVer1.3\\_おb_wav.frq',
 'D:/春歌ナナVer1.3\\_んb.wav',
 'D:/春歌ナナVer1.3\\_んb_wav.frq',
 'D:/春歌ナナVer1.3\\あ.wav',
 'D:/春歌ナナVer1.3\\あ_wav.frq',
 'D:/春歌ナナVer1.3\\い.wav',
 'D:/春歌ナナVer1.3\\い_wav.frq',
 'D:/春歌ナナVer1.3\\いぇ.wav',
 'D:/春歌ナナVer1.3\\いぇ_wav.frq',
 'D:/春歌ナナVer1.3\\う.wav',
 'D:/春歌ナナVer1.3\\う_wav.frq',
 'D:/春歌ナナVer1.3\\うぃ.wav',
 'D:/春歌ナナVer1.3\\うぃ_wav.frq',
 'D:/春歌ナナVer1.3\\うぇ.wav',
 'D:/春歌ナナVer1.3\\うぇ_wav.frq',
 'D:/春歌ナナVer1.3\\うぉ.wav',
 'D:/春歌ナナVer1.3\\うぉ_wav.frq',
 'D:/春歌ナナVer1.3\\え.wav',
 'D:/春歌ナナVer1.3\\え_wav.frq',
 'D:/春歌ナナVer1.3\\お.wav',
 'D:/春歌ナナVer1.3\\お_wav.frq',
 'D:/春歌

In [6]:
# delete_files(r'D:\春歌ナナVer1.3\New\*')

In [7]:
for filename in lis:
    shutil.copy(filename,new_dest)

In [8]:
pron_text="""あa
いi
うu
えe
おo
かka
きki
くku
けke
こko
さsa
しshi
すsu
せse
そso
たta
ちchi
つtsu
てte
とto
なna
にni
ぬnu
ねne
のno
はha
ひhi
ふfu
へhe
ほho
まma
みmi
むmu
めme
もmo
やya
（い）(i)
ゆyu
（え）(e)
よyo
らra
りri
るru
れre
ろro
わwa
をwo
んn
がga
ぎgi
ぐgu
げge
ごgo
ざza
じji
ずzu
ぜze
ぞzo
だda
ぢdi
づdu
でde
どdo
ばba
びbi
ぶbu
べbe
ぼbo
ぱpa
ぴpi
ぷpu
ぺpe
ぽpo
ヴv
"""
kana=[]
alpha=[]
for line in pron_text.splitlines():
    if (line[0] != '（'):
        kana.append(line[0])
        alpha.append(line[1:])
        
table=dict((key,value) for key,value in zip(kana,alpha))

In [9]:
def get_pronunciation(kana):
    #单假名词典
    global table,full_table
    #长音
    def match_long_pattern(roma):
        long_pron=['aa','ii','uu','ee','ei','oo','ou']
        return roma in long_pron
    #拗音（列表中为小写）
    spe_dict=dict((k,v) for k,v in zip(list('ぁぃぅぇぉゃゅょ'), list('aiueoauo')))
    keep=None
    
    #气息音
    if kana[0] == '_':
        keep=kana
        kana="".join(list(filter(lambda x: x in table.keys(), kana)))
    
    if kana and kana[0] in table.keys():
        if len(kana) == 1 or name[1] not in spe_dict.keys():
            pron=table[kana[0]]
            print(pron)
        else:
            pron=table[kana[0]]
            if len(pron) == 1:
                pron=pron+spe_dict[kana[1]]
            elif len(pron) == 2:
                #长音
                if match_long_pattern(pron[1]+spe_dict[kana[1]]):
                    pron=pron+spe_dict[kana[1]]
                elif kana[0] == 'ふ':
                    pron=pron[:-1]+spe_dict[kana[1]]
                elif kana[0] in list('すず'):
                    pron=pron+spe_dict[kana[1]]
                else:
                    pron=pron[:-1]+'y'+spe_dict[kana[1]]
            else:
                pron=pron[:-1]+spe_dict[kana[1]]
            print(pron,'\tspe')
            full_table[kana]=pron
            
        if keep is not None:
            pron=keep.replace(kana,pron)
            
        return True,pron
    else:
        return False,kana

In [10]:
#确认无误后修改NDEBUG为True
NDEBUG=False

full_table=table
wavs=glob(new_dest+'/*.wav')
for wav in wavs:
    freq_name=wav.replace('.wav','_wav.frq')
    name=os.path.splitext(os.path.split(wav)[1])[0]
    print(wav,'\t',name,end='\t')
    
    success,pron=get_pronunciation(name)
    
    if success and NDEBUG:
        def rename_wav(fname,kana,alpha):
            dst=fname.replace(kana,alpha)
            if not os.path.exists(dst):
                shutil.move(fname, dst)
            else:
                print('file '+dst+' already exits,skip it.')

        if os.path.exists(wav) and os.path.exists(freq_name):
            rename_wav(wav,name,pron)
            rename_wav(freq_name,name,pron)
    else:
        print()

D:\春歌ナナVer1.3\New\_あb.wav 	 _あb	a

D:\春歌ナナVer1.3\New\_いb.wav 	 _いb	i

D:\春歌ナナVer1.3\New\_うb.wav 	 _うb	u

D:\春歌ナナVer1.3\New\_えb.wav 	 _えb	e

D:\春歌ナナVer1.3\New\_おb.wav 	 _おb	o

D:\春歌ナナVer1.3\New\_んb.wav 	 _んb	n

D:\春歌ナナVer1.3\New\あ.wav 	 あ	a

D:\春歌ナナVer1.3\New\い.wav 	 い	i

D:\春歌ナナVer1.3\New\いぇ.wav 	 いぇ	ie 	spe

D:\春歌ナナVer1.3\New\う.wav 	 う	u

D:\春歌ナナVer1.3\New\うぃ.wav 	 うぃ	ui 	spe

D:\春歌ナナVer1.3\New\うぇ.wav 	 うぇ	ue 	spe

D:\春歌ナナVer1.3\New\うぉ.wav 	 うぉ	uo 	spe

D:\春歌ナナVer1.3\New\え.wav 	 え	e

D:\春歌ナナVer1.3\New\お.wav 	 お	o

D:\春歌ナナVer1.3\New\か.wav 	 か	ka

D:\春歌ナナVer1.3\New\が.wav 	 が	ga

D:\春歌ナナVer1.3\New\き.wav 	 き	ki

D:\春歌ナナVer1.3\New\きぇ.wav 	 きぇ	kye 	spe

D:\春歌ナナVer1.3\New\きゃ.wav 	 きゃ	kya 	spe

D:\春歌ナナVer1.3\New\きゅ.wav 	 きゅ	kyu 	spe

D:\春歌ナナVer1.3\New\きょ.wav 	 きょ	kyo 	spe

D:\春歌ナナVer1.3\New\ぎ.wav 	 ぎ	gi

D:\春歌ナナVer1.3\New\ぎぇ.wav 	 ぎぇ	gye 	spe

D:\春歌ナナVer1.3\New\ぎゃ.wav 	 ぎゃ	gya 	spe

D:\春歌ナナVer1.3\New\ぎゅ.wav 	 ぎゅ	gyu 	spe

D:\春歌ナナVer1.3\New\ぎょ.wav 	 ぎょ	gyo 	spe

D:\春歌ナナVer1.3\New\く.wav 	 く	ku


In [11]:
with open(new_dest+'/oto-clean.ini','r',encoding='utf-8') as f:
    ctx=f.read()

In [12]:
keys=list(full_table.keys())
keys.sort(key=len,reverse=True)
re=[]
for line in ctx.splitlines():
    #不保证连续音正确性，只适合单独音
    for k in keys:
        line=line.replace(k,full_table[k])
    print(line)
    re.append(line)
    
with open(new_dest+'/oto.ini','w',encoding='utf-8') as f:
    f.write('\n'.join(re))

﻿#Charset:UTF8
～.wav=,54,99,70,21,10
a.wav=,31,88,90,16,10
i.wav=,16,66,54,15,4
ie.wav=,95,144,62,36,20
u.wav=,29,112,39,19,14
ui.wav=,65,139,93,48,19
ue.wav=,62,129,112,43,22
uo.wav=,78,67,99,31,13
e.wav=,31,84,49,16,7
o.wav=wo,30,150,46,14,8
ka.wav=,51,72,105,16,-11
ga.wav=,35,74,92,21,6
ki.wav=,49,70,69,39,5
kye.wav=,57,84,79,27,8
kya.wav=,33,124,45,39,6
kyu.wav=,35,113,50,49,7
kyo.wav=,43,120,56,38,11
gi.wav=,33,129,38,35,11
gye.wav=,69,120,75,33,8
gya.wav=,56,96,60,30,10
gyu.wav=,64,82,76,32,10
gyo.wav=,73,112,88,32,12
ku.wav=,39,122,45,34,7
gu.wav=,23,84,39,35,11
ke.wav=,22,88,38,37,7
ge.wav=,46,76,50,21,9
ko.wav=,27,61,33,30,7
go.wav=,53,65,68,26,12
sa.wav=,104,141,51,82,20
za.wav=,43,113,49,38,12
shi.wav=,132,145,61,110,32
she.wav=,90,164,53,86,35
sha.wav=,141,231,37,144,79
shu.wav=,97,163,47,103,45
sho.wav=,86,165,40,88,42
ji.wav=di,38,110,61,45,22
jye.wav=,84,124,76,43,12
jya.wav=,80,115,82,35,9
jyu.wav=,44,104,76,50,15
jyo.wav=,48,109,57,39,15
su.wav=,93,133,42,97,49
sui.wav